# Transformación

In [54]:
import pandas as pd
import numpy as np
import os 
import glob

In [55]:
MAPA_TR = {
    "01_VOLTAJE_AC_DEL_SISTEMA_L1_L2": "voltaje_ac_l1_l2",
    "02_VOLTAJE_AC_DEL_SISTEMA_L2-L3": "voltaje_ac_l2_l3",
    "03_VOLTAJE_AC_DEL_SISTEMA_L3-L1": "voltaje_ac_l3_l1",
    "11_CORRIENTE_AC_DE_LA_CARGA_L1": "corriente_ac_l1",
    "12_CORRIENTE_AC_DE_LA_CARGA_L2": "corriente_ac_l2",
    "13_CORRIENTE_AC_DE_LA_CARGA_L3": "corriente_ac_l3",
    "14_POTENCIA_ACTIVA_DE_LA_CARGA": "potencia_activa_kw",
    "15_POTENCIA_REACTIVA_DE_LA_CARGA": "potencia_reactiva_kvar",
    "16_POTENCIA_APARENTE_DE_LA_CARGA": "potencia_aparente_kva", 
    "17_FACTOR_DE_POTENCIA_DE_LA_CARGA": "factor_potencia"
}

MAPA_ML = {
    "ANALOG_INPUT_-_ML_CURRENT_AC_R": "corriente_ac_r",
    "ANALOG_INPUT_-_ML_CURRENT_AC_S": "corriente_ac_s",
    "ANALOG_INPUT_-_ML_CURRENT_AC_T": "corriente_ac_t",
    "ANALOG_INPUT_-_ML_VOLTAGE_AC_R-S": "voltaje_ac_rs",
    "ANALOG_INPUT_-_ML_VOLTAGE_AC_S-T": "voltaje_ac_st",
    "ANALOG_INPUT_-_ML_VOLTAGE_AC_T-R": "voltaje_ac_tr",
    "ANALOG_INPUT_C_SALA_S01": "temp_sala_s01",
    "ANALOG_INPUT_C_SALA_S02": "temp_sala_s02"
}

MAPA_RECT1 = {
    "01_-_VOLTAJE_AC_DEL_SISTEMA" : "voltaje_ac_vs",
    "02_-_VOLTAJE_DC_DEL_SISTEMA": "voltaje_dc_vs",
    "03_-_CORRIENTE_DC_DEL_SISTEMA": "corriente_dc_cs",
    "11_-_CORRIENTE_DC_DE_LA_CARGA": "corriente_dc_carga"
}

MAPA_RECT2 = {
    "01_-_VOLTAJE_AC_DEL_SISTEMA" : "voltaje_ac_vs",
    "02_-_VOLTAJE_DC_DEL_SISTEMA": "voltaje_dc_vs",
    "03_-_CORRIENTE_DC_DEL_SISTEMA": "corriente_dc_cs",
    "11_-_CORRIENTE_DC_DE_LA_CARGA": "corriente_dc_carga"
}



In [56]:
def limpiar_fecha(fecha_str):
    """
    TR - IDEO CALI,"01 - VOLTAJE AC DEL SISTEMA L1-L2","216.0","V","1/1/2025, 12:50:03 a.Â m."

    Arregla el formato y elimina caracteres de codificación corruptos (Â).
    Entrada: '1/1/2025, 12:00:00 a.Â m.' o '1/1/2025, 12:00:00 a.\xa0m.'
    Salida: '1/1/2025 12:00:00 AM'
    """

    if not isinstance(fecha_str, str):
        return str(fecha_str)
    

    limpia = fecha_str.replace("Â", "") \
                      .replace("\xa0", " ") \
                      .replace(",", "") 
    
    limpia = limpia.lower() \
                   .replace("a. m.", "AM") \
                   .replace("p. m.", "PM") \
                   .replace("a.m.", "AM") \
                   .replace("p.m.", "PM") \
                   .replace("am", "AM") \
                   .replace("pm", "PM")
    
    return limpia.strip() # Eliminar espacios 

In [57]:
def procesar_carpeta_sensor(ruta, nombre_carpeta, nombre_columna_db):
    """
    Lee los 12 meses de una carpeta, limpia y consolida.
    """

    ruta_busqueda = os.path.join(ruta, nombre_carpeta, "*.csv") # O *.cvs si tienen error de extension
    archivos = glob.glob(ruta_busqueda)
    
    if not archivos:
        print(f"ADVERTENCIA: No se encontraron archivos en {nombre_carpeta}")
        return None

    print(f"Procesando {nombre_carpeta} ({len(archivos)} archivos)...")
    
    dfs_meses = []
    
    for archivo in archivos:
        try:
            # Leemos el CSV. Importante: dtype=str para no romper la lectura con 'Unplugged'
            df_temp = pd.read_csv(archivo, encoding='latin-1', dtype=str) # Probamos latin-1 por los caracteres raros
            
            df_temp['Time_Clean'] = df_temp['Time'].apply(limpiar_fecha)
            df_temp['timestamp'] = pd.to_datetime(df_temp['Time_Clean'], format='%d/%m/%Y %I:%M:%S %p', errors='coerce')
            
            # Convertimos Value a números. Los textos se vuelven NaN.
            df_temp['valor_numerico'] = pd.to_numeric(df_temp['Value'], errors='coerce')
            
            #Eliminar filas con NaN
            df_temp = df_temp.dropna(subset=['valor_numerico', 'timestamp'])
            
            # Seleccionamos solo lo que sirve (ya limpio)
            df_util = df_temp[['timestamp', 'valor_numerico']].copy()
            
            dfs_meses.append(df_util)
            
        except Exception as e:
            print(f"Error leyendo {os.path.basename(archivo)}: {e}")

    if not dfs_meses:
        return None

    # Unir enero y diciembre 
    df_anual = pd.concat(dfs_meses, ignore_index=True)
    
    # Ordenar por fecha
    df_anual = df_anual.sort_values('timestamp')
    df_anual = df_anual.set_index('timestamp')
    

    # *************************RESAMPLING (Normalización de tiempo)*******************************************************************
    # Como los datos vienen a 12:01:59 y otros a 12:05:00, promediamos cada 10 minutos
    # Esto alinea los datos y reduce el ruido.
    df_resampled = df_anual.resample('10min').mean()
    
    # Renombramos la columna 'valor_numerico' al nombre real del sensor (ej: voltaje_ac_l1_l2)
    df_resampled.columns = [nombre_columna_db]
    
    return df_resampled

In [58]:
ELEMENTOS_DCE = [
    ("TR",    "./Datos/DCE_DATOS_2025/TR",    MAPA_TR),
    ("ML",    "./Datos/DCE_DATOS_2025/ML",    MAPA_ML),
    ("Rect1", "./Datos/DCE_DATOS_2025/Rect1", MAPA_RECT1),
    ("Rect2", "./Datos/DCE_DATOS_2025/Rect2", MAPA_RECT2)
]

for nombre_dev, ruta_base, mapa_sensores in ELEMENTOS_DCE:
    print(f"\n" + "="*50)
    print(f"PROCESANDO DISPOSITIVO: {nombre_dev}")
    print(f"="*50)
    
    dfs_del_dispositivo = []
    

    for carpeta_sensor, columna_db in mapa_sensores.items():            
        df_sensor = procesar_carpeta_sensor(ruta_base, carpeta_sensor, columna_db)
        
        if df_sensor is not None:
            dfs_del_dispositivo.append(df_sensor)
    
    if dfs_del_dispositivo:
        print(f"Unificando sensores de {nombre_dev}...")
        df_final = pd.concat(dfs_del_dispositivo, axis=1)
        
        # Limpieza final (interpolación o reset index)
        df_final = df_final.reset_index()
        
        # Guardar
        carpeta_destino = os.path.join("Datos", "Historico")
        os.makedirs(carpeta_destino, exist_ok=True)

        nombre_archivo = f"consolidado_{nombre_dev}_2025.csv"
        ruta_completa = os.path.join(carpeta_destino, nombre_archivo)

        df_final.to_csv(ruta_completa, index=False)

        print(f"Guardado {nombre_archivo} ({len(df_final)} filas)")
    else:
        print(f"ADVERTENCIA: No se generaron datos para {nombre_dev}")


PROCESANDO DISPOSITIVO: TR
Procesando 01_VOLTAJE_AC_DEL_SISTEMA_L1_L2 (12 archivos)...
Procesando 02_VOLTAJE_AC_DEL_SISTEMA_L2-L3 (12 archivos)...
Procesando 03_VOLTAJE_AC_DEL_SISTEMA_L3-L1 (12 archivos)...
Procesando 11_CORRIENTE_AC_DE_LA_CARGA_L1 (12 archivos)...
Procesando 12_CORRIENTE_AC_DE_LA_CARGA_L2 (12 archivos)...
Procesando 13_CORRIENTE_AC_DE_LA_CARGA_L3 (12 archivos)...
Procesando 14_POTENCIA_ACTIVA_DE_LA_CARGA (12 archivos)...
Procesando 15_POTENCIA_REACTIVA_DE_LA_CARGA (12 archivos)...
Procesando 16_POTENCIA_APARENTE_DE_LA_CARGA (12 archivos)...
Procesando 17_FACTOR_DE_POTENCIA_DE_LA_CARGA (12 archivos)...
Unificando sensores de TR...
Guardado consolidado_TR_2025.csv (52560 filas)

PROCESANDO DISPOSITIVO: ML
Procesando ANALOG_INPUT_-_ML_CURRENT_AC_R (12 archivos)...
Procesando ANALOG_INPUT_-_ML_CURRENT_AC_S (12 archivos)...
Procesando ANALOG_INPUT_-_ML_CURRENT_AC_T (12 archivos)...
Procesando ANALOG_INPUT_-_ML_VOLTAGE_AC_R-S (12 archivos)...
Procesando ANALOG_INPUT_-_ML_V

# Base de datos

In [59]:
from sqlalchemy import create_engine, text
import getpass

In [60]:
USER = 'root'
HOST = 'localhost'
PORT = '3306'
DB_NAME = 'NODO_IDEO'
PASS = "admin"
#PASS = getpass.getpass("contraseña de MySQL: ")

In [61]:
# Cadena de conexión SIN base de datos (para poder crearla)
URL_SERVER = f"mysql+pymysql://{USER}:{PASS}@{HOST}:{PORT}"
# Cadena de conexión CON base de datos (para crear tablas)
URL_DB = f"{URL_SERVER}/{DB_NAME}"

# 1. Conectar al servidor para crear la BD
engine_server = create_engine(URL_SERVER)

with engine_server.connect() as conn:
    # Crea la BD con utf8mb4
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {DB_NAME} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"))
    print(f"Base de datos '{DB_NAME}' verificada.")


Base de datos 'NODO_IDEO' verificada.


In [62]:
engine_db = create_engine(URL_DB)

with engine_db.connect() as conn:

    sql_tr = """
    CREATE TABLE IF NOT EXISTS tr_historico (
        timestamp DATETIME NOT NULL,
        voltaje_ac_l1_l2 FLOAT,
        voltaje_ac_l2_l3 FLOAT,
        voltaje_ac_l3_l1 FLOAT,
        corriente_ac_l1 FLOAT,
        corriente_ac_l2 FLOAT,
        corriente_ac_l3 FLOAT,
        potencia_activa_kw FLOAT,
        potencia_reactiva_kvar FLOAT,
        potencia_aparente_kva FLOAT,
        factor_potencia FLOAT,
        PRIMARY KEY (timestamp)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    
    
    sql_ml = """
    CREATE TABLE IF NOT EXISTS ml_historico (
        timestamp DATETIME NOT NULL,
        corriente_ac_r FLOAT,
        corriente_ac_s FLOAT,
        corriente_ac_t FLOAT,
        voltaje_ac_rs FLOAT,
        voltaje_ac_st FLOAT,
        voltaje_ac_tr FLOAT,
        temp_sala_s01 FLOAT,
        temp_sala_s02 FLOAT,
        PRIMARY KEY (timestamp)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    
    sql_rect1 = """
    CREATE TABLE IF NOT EXISTS rect1_historico (
        timestamp DATETIME NOT NULL,
        voltaje_ac_vs FLOAT,
        voltaje_dc_vs FLOAT,
        corriente_dc_cs FLOAT,
        corriente_dc_carga FLOAT,
        PRIMARY KEY (timestamp)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """

    sql_rect2 = """
    CREATE TABLE IF NOT EXISTS rect2_historico (
        timestamp DATETIME NOT NULL,
        voltaje_ac_vs FLOAT,
        voltaje_dc_vs FLOAT,
        corriente_dc_cs FLOAT,
        corriente_dc_carga FLOAT,
        PRIMARY KEY (timestamp)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """

    

    conn.execute(text(sql_tr))
    conn.execute(text(sql_ml))
    conn.execute(text(sql_rect1))
    conn.execute(text(sql_rect2))


# Carga 

In [63]:
import pandas as pd
import os
from sqlalchemy import text

In [64]:
def cargar_datos_sql(engine_db):

    carpeta_origen = os.path.join("Datos", "Historico")

    mapa_carga = {
        "consolidado_TR_2025.csv":    "tr_historico",
        "consolidado_ML_2025.csv":    "ml_historico",
        "consolidado_Rect1_2025.csv": "rect1_historico",
        "consolidado_Rect2_2025.csv": "rect2_historico"
    }

    for archivo_csv, nombre_tabla in mapa_carga.items():
        ruta_completa = os.path.join(carpeta_origen, archivo_csv)
        
        # Verificamos si el archivo existe antes de intentar leerlo
        if not os.path.exists(ruta_completa):
            print(f" AVISO: No se encontró {archivo_csv}, saltando...")
            continue

        try:
            df = pd.read_csv(ruta_completa)
            
            if 'timestamp' in df.columns:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            print(f" Subiendo {len(df)} registros a la tabla '{nombre_tabla}'...")
            
            # if_exists='append': Agrega los datos a la tabla que ya creaste.
            # index=False: No subir el índice de pandas (0, 1, 2...) como columna.
            # chunksize=1000: Sube de a 1000 filas para no saturar la red/memoria.
            df.to_sql(
                name=nombre_tabla,
                con=engine_db,
                if_exists='append', 
                index=False,
                chunksize=1000
            )
            
            print(f"Carga completada en '{nombre_tabla}'.")

        except Exception as e:
            print(f" ERROR cargando {nombre_tabla}: {e}")
            # Tip: Si el error es "Duplicate entry", significa que ya corriste esto antes.

    print("\n PROCESO DE CARGA FINALIZADO.")

In [ ]:
engine_db = create_engine(URL_DB)
cargar_datos_sql(engine_db)